In [1]:
from google.colab import files
uploaded = files.upload()

Saving SMSSpamCollection to SMSSpamCollection


In [13]:
import pandas as pd
import io
messages = pd.read_csv(io.BytesIO(uploaded['SMSSpamCollection']), sep='\t',
                           names=["label", "message"])

In [3]:
#Data cleaning and preprocessing
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['message'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [5]:
# Creating the Bag of Words model
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()

In [6]:
y=pd.get_dummies(messages['label'])
y=y.iloc[:,1].values

In [7]:
# Train Test Split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [8]:
# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred=spam_detect_model.predict(X_test)

In [9]:
from sklearn.metrics import accuracy_score
Score = accuracy_score(y_test,y_pred)
print(Score)

0.9856502242152466


In [11]:
from sklearn.metrics import classification_report
Report = classification_report(y_test,y_pred)
print(Report)

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       955
           1       0.94      0.96      0.95       160

    accuracy                           0.99      1115
   macro avg       0.97      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



# ***Using Support Vector Machines***

In [14]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
Parameter_svc=[{'kernel': ['rbf','poly'],
               'degree': [3,4,5]}]
scores = ['accuracy']

clf_svc = GridSearchCV(SVC(), Parameter_svc, scoring='accuracy', verbose=2, cv=5   
                         )
clf_svc.fit(X_train,y_train)


print(clf_svc.best_params_)

y_pred1 = clf_svc.predict(X_test)

accuracy_score(y_pred1,y_test)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] degree=3, kernel=rbf ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ............................. degree=3, kernel=rbf, total=  21.4s
[CV] degree=3, kernel=rbf ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   21.4s remaining:    0.0s


[CV] ............................. degree=3, kernel=rbf, total=  21.4s
[CV] degree=3, kernel=rbf ............................................
[CV] ............................. degree=3, kernel=rbf, total=  21.7s
[CV] degree=3, kernel=rbf ............................................
[CV] ............................. degree=3, kernel=rbf, total=  22.0s
[CV] degree=3, kernel=rbf ............................................
[CV] ............................. degree=3, kernel=rbf, total=  22.1s
[CV] degree=3, kernel=poly ...........................................
[CV] ............................ degree=3, kernel=poly, total=  45.2s
[CV] degree=3, kernel=poly ...........................................
[CV] ............................ degree=3, kernel=poly, total=  42.0s
[CV] degree=3, kernel=poly ...........................................
[CV] ............................ degree=3, kernel=poly, total=  44.1s
[CV] degree=3, kernel=poly ...........................................
[CV] .

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 17.4min finished


{'degree': 3, 'kernel': 'rbf'}


0.9838565022421525

In [15]:
print(accuracy_score(y_pred1,y_test))

0.9838565022421525


In [16]:
Report = classification_report(y_test,y_pred1)
print(Report)

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       955
           1       1.00      0.89      0.94       160

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.97      1115
weighted avg       0.98      0.98      0.98      1115

